In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('data/DataSet_Lakas_Regression.csv')

In [6]:
df = df.loc[df.property_area > 0]
df = df.loc[(df.price_created_at >= 5) & (df.price_created_at <= 100)]
df['target'] = df.price_created_at/df.property_area * 1000000

In [8]:
df.head()

,county,city,postcode,property_type,property_subtype,property_condition_type,property_floor,building_floor_count,view_type,orientation,...,room_cnt,small_room_cnt,created_at,property_area,balcony_area,price_created_at,ad_view_cnt,active_days,nr,target
0,Budapest,Budapest XII.,NaN,flat,brick flat (for sale),good,1,NaN,street view,NaN,...,2.0,1.0,2015-02-09,65.0,0.0,23.5,605.0,119.0,4,361538.461538
1,Budapest,Budapest I.,1016.0,flat,brick flat (for sale),novel,2,NaN,street view,NaN,...,1.0,1.0,2015-02-09,45.0,0.0,20.0,49.0,25.0,12,444444.444444
2,Budapest,Budapest XVI.,1164.0,flat,brick flat (for sale),novel,1,NaN,garden view,NaN,...,2.0,1.0,2015-02-09,60.0,0.0,22.0,77.0,77.0,14,366666.666667
3,Budapest,Budapest X.,NaN,flat,brick flat (for sale),good,4,NaN,garden view,NaN,...,2.0,0.0,2015-02-09,55.0,4.0,11.0,139.0,18.0,21,200000.000000
4,Budapest,Budapest XVIII.,1181.0,flat,prefabricated panel flat (for sale),renewed,6,NaN,NaN,NaN,...,2.0,1.0,2015-02-09,60.0,3.0,10.2,176.0,69.0,31,170000.000000


In [11]:
filtarable_parameters = ['county', 'postcode', 'ad_view_cnt', 'property_type', 'nr', 'created_at', 'price_created_at']
df = df.drop(filtarable_parameters, axis=1)

In [12]:
df = df.dropna()

In [13]:
df.elevator_type = df.elevator_type.apply(lambda x: 1 if x == 'yes' else 0)
df.garden_access = df.garden_access.apply(lambda x: 1 if x == 'yes' else 0)

In [14]:
df.property_condition_type.unique()

array(['novel', 'medium', 'good', 'renewed', 'new_construction',
       'to_be_renovated', 'missing_info', 'can_move_in',
       'under_construction'], dtype=object)

In [18]:
property_cond = ['missing_info', 'to_be_renovated', 'medium', 'good', 'under_construction', 'can_move_in', 'novel', 'renewed', 'new_construction']
le = preprocessing.LabelEncoder()
le.fit(property_cond)
df.property_condition_type = le.transform(df.property_condition_type.values)

In [25]:
columnes_to_encode = ['city', 'property_subtype', 'property_floor', 'building_floor_count', 'view_type', 'orientation', 'heating_type']
for column in columnes_to_encode:
    one_hot = pd.get_dummies(df[column])
    one_hot.columns = one_hot.columns + '_' + column
    df = df.drop(column, axis=1)
    df = df.join(one_hot)

In [30]:
target = ['target']
y = df[target].values
x = df.drop(target, axis=1).values

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [34]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
rg = linear_model.Ridge(alpha=0.01)
lasso = linear_model.Lasso()
lr.fit(x_train, y_train)
rg.fit(x_train, y_train)
lasso.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [36]:
# Create the MAPE function for evaluation
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mapelr= mean_absolute_percentage_error(y_test, lr.predict(x_test))
maperg= mean_absolute_percentage_error(y_test, rg.predict(x_test))
mapelasso= mean_absolute_percentage_error(y_test, lasso.predict(x_test))
print(f'lr: {mapelr}, rg: {maperg}, lasso {mapelasso}')

lr: 18.94311103822175, rg: 18.94306443485037, lasso 43.048507988194174


In [39]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(lr, x, y, cv=10, scoring='neg_mean_squared_error')
print(cv_results['test_score'])

[-1.35489299e+10 -2.34655320e+10 -9.61828609e+09 -4.08165820e+10
 -7.37768563e+09 -1.40787084e+10 -2.46373018e+10 -1.11727195e+10
 -9.52781130e+09 -1.42769636e+10]


Onallo feladat

In [40]:
df = pd.read_csv('data/DataSet_student.csv')

In [44]:
df['sex_b'] = df.sex.apply(lambda x: 1 if 'M' else 0)
df['school_b'] = df.school.apply(lambda x: 1 if 'GP' else 0)
df['address_b'] = df.address.apply(lambda x: 1 if 'U' else 0)
df['Pstatus_b'] = df.Pstatus.apply(lambda x: 1 if 'T' else 0)

In [48]:
target=['G3']
x = df[target].values
y = df[['sex_b', 'school_b', 'address_b', 'Pstatus_b', 'Medu']].values

In [51]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(lr, x, y, cv=10, scoring='neg_mean_squared_error')
print(cv_results['test_score'])
print(cv_results['test_score'].mean())

[-0.19187337 -0.22314634 -0.18797867 -0.31235588 -0.2280347  -0.18287152
 -0.22860586 -0.27012877 -0.20553581 -0.29649147]
-0.23270224033160775


In [56]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,sex_b,school_b,address_b,Pstatus_b
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.0,395.0,395.0,395.0
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.067342,1.0,1.0,1.0,1.0
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.584270,0.0,0.0,0.0,0.0
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,-0.750000,1.0,1.0,1.0,1.0
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000,1.0,1.0,1.0,1.0
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,10.500000,1.0,1.0,1.0,1.0
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,13.250000,1.0,1.0,1.0,1.0
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,19.500000,1.0,1.0,1.0,1.0
